# Construct a virtual cloud for Galaxy + SLURM cluster


AWSクラウドの上にGalaxyクラスタ環境の VC (Virtual Cloud) を作成する。
- Galaxyサーバとそのジョブ実行先となる計算ノード群をSLURMクラスタとして構成する。
- RNA-Seq用マッピングツールのHISAT2をGalaxyツールとして登録し、SLURMノード上で実行可能な環境を構築する。

![galaxy-slurm-cluster](media/galaxy-slurm.png)

| VC Node | Cloud (type) | Role |
|:--------|:-------------|:----------------------|
| 1 | AWS EC2 (m4.large) | NFS, Galaxy, SLURM Controller |
| 2 | AWS EC2 (m4.2xlarge) | SLURM Node \#1 |
| 3 | AWS EC2 (m4.2xlarge) | SLURM Node \#2 |

----
| 本Notebookで使用する用語について | |
|:--|:--|
| VC (Virtual Cloud) | Dockerコンテナを使用して複数のクラウドにまたがって構築された一つの仮想システム環境 |
| VC Node | VCを構成する個々のノード。<br>各クラウドから確保したクラウドインスタンス（VMもしくはBaremetal）とその上で独占的に動作するBaseコンテナから構成される |
| VC Unit | 同質（同じクラウド・同じフレーバ・同じ Base コンテナ）な VC Node を一括して取り扱うための単位 |
| Base コンテナ | VC Node 上で独占的に動作する Docker コンテナ。<br>コンテナ内でDocker Engineが稼働し、Docker in Docker 構成でApplicationコンテナを起動することができる。 |
| Application コンテナ | Base コンテナ内で Docker in Docker により起動するコンテナ |
| Base ネットワーク | Base コンテナ間をL2接続する仮想ネットワーク <br> SINET L2VPNに直結しない商用クラウドなどはL3レベルで接続される |

# Create a new VC

新規にGalaxyクラスタ用 VC を作成する作業の流れは以下のようになる。
1. Galaxyサーバ用 VC Node 作成
2. SLURMクラスタ用 VC Node 作成
3. 各 VC Node への SSH 接続設定
4. 共有ストレージ（AWSクラウドでは EBS ボリューム）のマウント
5. Galaxyサーバアプリケーション（コンテナ）起動
6. SLURMクラスタ（コンテナ）起動
7. Galaxyツール（HISAT2）設定
8. ツール実行 ＆ Grafanaダッシュボードによる負荷状況確認

----
まず、最初に下図の赤枠・青枠で示した各VC Nodeを作成する。

![galaxy-slurm-cluster](media/galaxy-slurm-2.png)

## Configure VCC parameters

使用するVCコントローラ環境に合わせてパラメータを設定する。

In [1]:
# VCC API Access KEY
vcp_access_key = 'KEY_FOR_TEST001'

# VC Name
my_vc_name = "galaxy-cluster001"

# Galaxy server IP アドレス
# ※ 今回のHands-Onでは内部 DNS を使用せず、固定IPで Galaxy を構成する
galaxy_server_ip = '172.30.2.100'

# EBS Volume ID  [TODO] 別ファイルにして配る
# block_storage_id = "vol-0e9f4fa0c1a52430c"

## Initialize VCP SDK

VCP SDKを初期化する。

In [2]:
import sys; sys.path.append('/notebooks/notebook/')
from common import logsetting
from vcpsdk import vcpsdk
from vcpsdk import vcp_config

v = vcpsdk.VcpSDK(vcp_access_key, my_vc_name)
block_storage_id = vcp_config.block_storage_id

In [3]:
print(block_storage_id)

vol-0e9f4fa0c1a52430c


## Create a node for Galaxy

- GalaxyサーバおよびSLURMコントローラ用のノードを作成する。
- 共有ストレージもこのノードに置くため、Base コンテナとして NFS Server 機能をインストール済みのイメージを使用する。

![galaxy-slurm-cluster](media/galaxy-slurm-2-3.png)

In [4]:
#
# 作成する Node の Spec 情報を作成
#
spec = v.spec.find_spec("aws", "small")

# Node 台数を指定 （IPアドレスを指定して起動する場合は、アドレス個数に依存）
#spec.num_nodes = 1
spec.node_ips = [galaxy_server_ip]

# インスタンスタイプを強制指定
spec.instance_type = "m4.large"

# Base Container Image
spec.image = 'vcp/base-nfs:1.0'                 

# Base Container Options
spec.params_v = ['/var/lib/docker', '/dev:/dev']
spec.params_e = ['NFS_AUTOSTART=true']

# NFS で利用する共有ストレージ用 EBS Volume ID を指定
spec.volume_id = [block_storage_id]

#
# 作成した Spec 情報を利用して Server(Unit) を起動
#
nodes = v.unit.create("galaxy", spec, wait_for=True, verbose=0)
galaxy_node = nodes[0]

2017-12-04 16:27:51,732 - INFO - BOOTING ... 0 sec
2017-12-04 16:27:57,039 - INFO - BOOTING ... 5 sec
2017-12-04 16:28:02,363 - INFO - BOOTING ... 10 sec
2017-12-04 16:28:07,688 - INFO - BOOTING ... 15 sec
2017-12-04 16:28:12,996 - INFO - BOOTING ... 20 sec
2017-12-04 16:28:18,319 - INFO - BOOTING ... 25 sec
2017-12-04 16:28:23,629 - INFO - BOOTING ... 30 sec
2017-12-04 16:28:28,937 - INFO - BOOTING ... 35 sec
2017-12-04 16:28:34,314 - INFO - BOOTING ... 40 sec
2017-12-04 16:28:39,654 - INFO - BOOTING ... 45 sec
2017-12-04 16:28:45,019 - INFO - BOOTING ... 50 sec
2017-12-04 16:28:50,349 - INFO - BOOTING ... 55 sec
2017-12-04 16:28:55,682 - INFO - BOOTING ... 60 sec
2017-12-04 16:29:00,995 - INFO - BOOTING ... 65 sec
2017-12-04 16:29:06,349 - INFO - unit galaxy is RUNNING


## Create a node for a SLURM cluster

- SLURMクラスタの Compute ノードを作成する。
- ここで作成する2台のノードは、Galaxyジョブを実行するための計算ノードとなる。

![galaxy-slurm-cluster](media/galaxy-slurm-2-4.png)

In [5]:
#
# 作成する Node の Spec 情報を作成
#
spec = v.spec.find_spec("aws", "small")

# 作成する Node の個数を指定（今回の構成では計算ノードを2台）
spec.num_nodes = 2

# インスタンスタイプを強制指定
spec.instance_type = "m4.2xlarge"

# Base Container Image
spec.image = 'vcp/base-nfs:1.0'                 

# Base Container Options
spec.params_v = ['/var/lib/docker', '/dev:/dev']
spec.params_e = ['NFS_AUTOSTART=false']

#
# 作成したSpec情報を利用して、Server(VC Unit)を起動
#
nodes = v.unit.create("galaxy_compute", spec, wait_for=True, verbose=0)
slurm_nodes = nodes

2017-12-04 16:32:22,142 - INFO - BOOTING ... 0 sec
2017-12-04 16:32:27,603 - INFO - BOOTING ... 5 sec
2017-12-04 16:32:33,056 - INFO - BOOTING ... 10 sec
2017-12-04 16:32:38,534 - INFO - BOOTING ... 15 sec
2017-12-04 16:32:44,008 - INFO - BOOTING ... 20 sec
2017-12-04 16:32:49,475 - INFO - BOOTING ... 25 sec
2017-12-04 16:32:54,968 - INFO - BOOTING ... 30 sec
2017-12-04 16:33:00,517 - INFO - BOOTING ... 35 sec
2017-12-04 16:33:06,072 - INFO - BOOTING ... 40 sec
2017-12-04 16:33:11,594 - INFO - unit galaxy_compute is RUNNING


## Confirm the created VC information

現在の VC 情報を確認する。

### Show a list of launched nodes

起動した VC Node 群の情報を取得する。

In [ ]:
v.unit.df_node()

共有ストレージのボリューム情報（AWSクラウドではEBSの Volume ID）を確認する。

In [ ]:
print(galaxy_node)
for vol in galaxy_node.volumes:
    print(vol)

# Prepare SSH setting for VC nodes

各VC Nodeに対してSSH公開鍵認証でアクセスできるように準備する。


![galaxy-slurm-cluster](media/galaxy-slurm-3.png)

各VC Nodeに対してデフォルトのパスワード認証で接続し、公開鍵配布先ディレクトリを作成する。

In [ ]:
galaxy_server_ip = galaxy_node.cloud_instance_address
galaxy_compute_ips = []
for node in slurm_nodes:
    galaxy_compute_ips.append(node.cloud_instance_address)
    
!./setup_ssh_login.sh $galaxy_server_ip 1
for ip in galaxy_compute_ips:
    !./setup_ssh_login.sh $ip 1

各VC Nodeに対応する鍵ペアを作成し、公開鍵を配置する。

In [ ]:
!./setup_ssh_login.sh $galaxy_server_ip 2
for ip in galaxy_compute_ips:
    !./setup_ssh_login.sh $ip 2

ssh_config のエントリ名を用いて各VC Nodeにログインできるようにする。

In [10]:
SERVER_SSH_CONF = '''\
StrictHostKeyChecking no
Host galaxy-server
  User root
  HostName {gip}
  Port 10022
  IdentityFile ssh/id_{gip}

'''

COMPUTE_SSH_CONF = '''\
Host galaxy-compute-{id}
  User root
  HostName {cip}
  Port 10022
  IdentityFile ssh/id_{cip}

'''

# Galaxy node
ssh_config = SERVER_SSH_CONF.format(gip=galaxy_server_ip)

# SLURM nodes
num = 1
for ip in galaxy_compute_ips:
    config = COMPUTE_SSH_CONF.format(id=num, cip=ip)
    ssh_config += config
    num += 1

!echo "$ssh_config" > tmp/ssh_config.ga

In [11]:
print(ssh_config)

StrictHostKeyChecking no
Host galaxy-server
  User root
  HostName 172.30.2.100
  Port 10022
  IdentityFile ssh/id_172.30.2.100

Host galaxy-compute-1
  User root
  HostName 172.30.2.167
  Port 10022
  IdentityFile ssh/id_172.30.2.167

Host galaxy-compute-2
  User root
  HostName 172.30.2.59
  Port 10022
  IdentityFile ssh/id_172.30.2.59




# Setup Base containers

各VC NodeのBaseコンテナ機能をセットアップする。


![galaxy-slurm-cluster](media/galaxy-slurm-4.png)

## Mount the NFS volume

EBS ボリュームは、Galaxyサーバ用ノード起動時のSpec情報で記述した順に `/dev/xvd[f]` から割り当てられ、 `/dev/xvd[p]` までの最大12個を使用可能。

まず、最初に Galaxyサーバ用ノードで、EBSボリューム `/dev/xvdf1` を `/export` へマウントする。  
（`/export` は NFS サーバインストール済み Base コンテナの起動時に NFS のエクスポート・ディレクトリとして設定されている）

In [12]:
!ssh -F tmp/ssh_config.ga galaxy-server "mkdir -p /export && mount /dev/xvdf1 /export"
!ssh -F tmp/ssh_config.ga galaxy-server df -h /export

Filesystem                Size      Used Available Use% Mounted on
/dev/xvdf1               62.9G     11.8G     47.9G  20% /export


続いて、各SLURM NodeからGalaxyサーバ上のNFSボリュームをマウントする。

In [13]:
num=1
for ip in galaxy_compute_ips:
    !ssh -F tmp/ssh_config.ga galaxy-compute-$num "rpcbind && mkdir -p /export"
    !ssh -F tmp/ssh_config.ga galaxy-compute-$num mount -t nfs $galaxy_server_ip:/export /export
    num += 1

各SLURM NodeのNFSマウント状態を確認する。

In [14]:
num = 1
for ip in galaxy_compute_ips:
    !echo galaxy-compute-$num
    !ssh -F tmp/ssh_config.ga galaxy-compute-$num df -h /export
    num += 1

galaxy-compute-1
Filesystem                Size      Used Available Use% Mounted on
172.30.2.100:/export     62.9G     11.8G     47.9G  20% /export
galaxy-compute-2
Filesystem                Size      Used Available Use% Mounted on
172.30.2.100:/export     62.9G     11.8G     47.9G  20% /export


# Setup Application containers

各Baseコンテナ上でGalaxyサーバと計算ノード用の各コンテナをセットアップする。


![galaxy-slurm-cluster](media/galaxy-slurm-5.png)

## Configure hostname resolution

- SLURMクラスタを運用する場合、各ホストはIPアドレスではなくホスト名でアクセスできなければならない。  
  内部向けDNS Serviceを起動する方法もあるが、今回はクラスタ構成が固定のため /etc/hosts に記述することでDNS解決を行う。
- GalaxyおよびSLURM用 Applicationコンテナに設定する `/etc/hosts` の内容を作成する。  
  これを Applicationコンテナを `docker run` する際のオプションとして指定することができる。 

In [15]:
add_host_opt = '--add-host master:%s' % (galaxy_server_ip)
i = 1
galaxy_compute_nodes = v.unit.find_node("galaxy_compute")
for node in galaxy_compute_nodes:
    add_host_opt += ' --add-host c%d:%s' % (i, node.cloud_instance_address)
    i += 1

In [16]:
print(add_host_opt)

--add-host master:172.30.2.100 --add-host c1:172.30.2.167 --add-host c2:172.30.2.59


## Start the Galaxy container

今回のハンズオン環境ではGalaxyサーバの共有ストレージにあらかじめ以下の情報がセットアップされている。

1. Galaxy Server用データアーカイブが /export/ 直下に展開済み。
2. HISAT2用インデックスアーカイブが /export/genome/ 直下に展開済み。

Galaxyの公式Dockerイメージ  `bgruening/galaxy-stable` を起動する。  
  * `bgruening/galaxy-stable` イメージは起動時に Docker Hub から取得することもできるが、  
     今回のハンズオン環境では利用者毎の Application コンテナレジストリに登録しておいたものを利用する。

In [17]:
BOOT_GALAXY_TMPL = '''\
/usr/local/bin/docker run -d --net=host \
  {add_host_opt} \
  --hostname master --name galaxy\
  -v /export/:/export/ \
  -e "NONUSE=nodejs,proftp,reports,slurmctld,slurmd,condor" \
  -e "PROXY_PREFIX=/galaxy" \
  10.0.0.1:5001/galaxy-stable:20171018 \
  2>&1 | tail -n3
'''
boot_galaxy_cmd = BOOT_GALAXY_TMPL.format(add_host_opt=add_host_opt)

In [18]:
!ssh -F tmp/ssh_config.ga galaxy-server $boot_galaxy_cmd

Digest: sha256:64c2690ef5594b9aed69fff68d2eecf62ddbf8d5df23c73bb40ac7d4e0152f45
Status: Downloaded newer image for 10.0.0.1:5001/galaxy-stable:20171018
aac907dc6a1add61ff11c1e8ac76a95322dc3fbf230149645d7159f41e958a7c


起動が完了するまで数分程度を要する。以下の2つの worker に関するログが出力されていることを確認する。

- starting galaxy control worker for main.1
- starting galaxy control worker for main.2

In [20]:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker logs -t --tail 10 galaxy

2017-12-04T07:44:59.741353635Z galaxy.webapps.util DEBUG 2017-12-04 07:44:59,439 Enabling 'BatchMiddleware' middleware
2017-12-04T07:44:59.741357869Z galaxy.webapps.galaxy.buildapp DEBUG 2017-12-04 07:44:59,439 Prior to webapp return, Galaxy thread <_MainThread(MainThread, started 139638697883456)> is alive.
2017-12-04T07:44:59.741362191Z Mon Dec  4 07:44:59 2017 - WSGI app 0 (mountpoint='') ready in 15 seconds on interpreter 0x20250a0 pid: 287 (default app)
2017-12-04T07:44:59.741366064Z Mon Dec  4 07:44:59 2017 - *** uWSGI is running in multiple interpreter mode ***
2017-12-04T07:44:59.741369731Z Mon Dec  4 07:44:59 2017 - spawned uWSGI master process (pid: 287)
2017-12-04T07:44:59.741373748Z Mon Dec  4 07:44:59 2017 - spawned uWSGI worker 1 (pid: 399, cores: 4)
2017-12-04T07:44:59.741377639Z Mon Dec  4 07:44:59 2017 - spawned uWSGI worker 2 (pid: 402, cores: 4)
2017-12-04T07:44:59.741381853Z Mon Dec  4 07:44:59 2017 - *** Stats server enabled on 127.0.0.1:9191 fd: 12 ***
2017-12-04T

Galaxyサーバ上にあるSLURMコントローラ( `slurmctld` )を再起動する。
  * 共有ストレージ `/export/slurm.conf` にある事前に用意したSLURM設定ファイルを読み込むように設定する。

In [21]:
slurm_restart_cmd = '''\
pkill slurmctld
pkill munged
munged --key-file=/export/munge.key -f
slurmctld -L /home/galaxy/logs/slurmctld.log -f /export/slurm.conf
'''

!echo "$slurm_restart_cmd" > tmp/slurm_restart.sh

In [22]:
!scp -F tmp/ssh_config.ga tmp/slurm_restart.sh galaxy-server:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker cp slurm_restart.sh galaxy:/tmp/
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy sh /tmp/slurm_restart.sh

slurm_restart.sh                              100%  136     0.1KB/s   00:00    


現段階では、STATE が "`unk`" (unknown) のノードが2つ見えることを確認する。

In [23]:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy sinfo

PARTITION AVAIL  TIMELIMIT  NODES  STATE NODELIST
work*        up   infinite      2   unk* c[1-2]


`bgruening/galaxy-stable` のDockerイメージに `samtools` が含まれていないため、パッケージを追加でインストールする。
  * samtools が無いと HISAT2 実行時に Galaxy サーバが結果のBAMファイルを格納する際にエラーとなるため。
 

In [ ]:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy apt-get update
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy apt-get -y install samtools

## Start SLURM containers

Galaxy用に設定されたSLURM Computeノード向けコンテナとして `docker-galaxy-stable` で提供されているものが利用できる。
  * https://github.com/bgruening/docker-galaxy-stable/tree/master/test/slurm

今回はこれを事前にビルドしてApplicationコンテナレジストリに登録しておいたものを利用する。  
なお、このコンテナには、今回のハンズオンで実行するツールとして `HISAT2` がインストール済みである。

In [25]:
BOOT_SLURM_TMPL = '''\
/usr/local/bin/docker run -d --net=host \
  {add_host_opt} \
  --hostname c{id} --name slurm{id} \
  -v /export/:/export/ \
  10.0.0.1:5001/galaxy-slurm:20171027 \
  2>&1 | tail -n3
'''

num=1
for ip in galaxy_compute_ips:
    boot_slurm_cmd = BOOT_SLURM_TMPL.format(add_host_opt=add_host_opt, id=num)
    !echo "$boot_slurm_cmd" > tmp/slurm_bootstrap.sh
    !scp -F tmp/ssh_config.ga tmp/slurm_bootstrap.sh galaxy-compute-$num:
    !ssh -F tmp/ssh_config.ga galaxy-compute-$num sh ./slurm_bootstrap.sh
    num += 1

slurm_bootstrap.sh                            100%  236     0.2KB/s   00:00    
Digest: sha256:1ff7887a9fe424a70ee0cad364dcbf07262dd17cbbc76059e0205f8f0d6ff0cb
Status: Downloaded newer image for 10.0.0.1:5001/galaxy-slurm:20171027
0d7421858eac09087ca710aabbd9dc2bea63dcc92543c94502a03c4914bdae65
slurm_bootstrap.sh                            100%  236     0.2KB/s   00:00    
Digest: sha256:1ff7887a9fe424a70ee0cad364dcbf07262dd17cbbc76059e0205f8f0d6ff0cb
Status: Downloaded newer image for 10.0.0.1:5001/galaxy-slurm:20171027
9b86d2a500059a483abec8129576a76699a0b4ef14168504334b94e956286046


Galaxy サーバ (SLURM Controller側) から各計算ノードに構成されたSLURMクラスタの情報を取得する。

STATE が "`idle`" のノードが2つ表示されることを確認する。

In [26]:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy sinfo --long

Mon Dec  4 07:51:05 2017
PARTITION AVAIL  TIMELIMIT   JOB_SIZE ROOT SHARE     GROUPS  NODES       STATE NODELIST
work*        up   infinite 1-infinite   no YES:4        all      2        idle c[1-2]


SLURMノードの詳細情報を確認する。

In [27]:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy scontrol show node c[1-2]

NodeName=c1 Arch=x86_64 CoresPerSocket=1
   CPUAlloc=0 CPUErr=0 CPUTot=8 CPULoad=0.07 Features=(null)
   Gres=(null)
   NodeAddr=c1 NodeHostName=c1
   OS=Linux RealMemory=20000 AllocMem=0 Sockets=8 Boards=1
   State=IDLE ThreadsPerCore=1 TmpDisk=0 Weight=1
   BootTime=2017-12-04T07:32:46 SlurmdStartTime=2017-12-04T07:50:33
   CurrentWatts=0 LowestJoules=0 ConsumedJoules=0
   ExtSensorsJoules=n/s ExtSensorsWatts=0 ExtSensorsTemp=n/s
   

NodeName=c2 Arch=x86_64 CoresPerSocket=1
   CPUAlloc=0 CPUErr=0 CPUTot=8 CPULoad=0.04 Features=(null)
   Gres=(null)
   NodeAddr=c2 NodeHostName=c2
   OS=Linux RealMemory=20000 AllocMem=0 Sockets=8 Boards=1
   State=IDLE ThreadsPerCore=1 TmpDisk=0 Weight=1
   BootTime=2017-12-04T07:32:47 SlurmdStartTime=2017-12-04T07:50:58
   CurrentWatts=0 LowestJoules=0 ConsumedJoules=0
   ExtSensorsJoules=n/s ExtSensorsWatts=0 ExtSensorsTemp=n/s
   



## Configure Galaxy job runner

Galaxyがジョブ実行プラグイン(Job Runner)として `SlurmJobRunner` を使うように設定する。

In [ ]:
# https://github.com/bgruening/docker-galaxy-stable/blob/master/test/slurm/job_conf.xml
job_conf = '''\
<?xml version="1.0"?>
<job_conf>
    <plugins workers="2">
        <plugin id="slurm" type="runner" load="galaxy.jobs.runners.slurm:SlurmJobRunner">
            <param id="drmaa_library_path">/usr/lib/slurm-drmaa/lib/libdrmaa.so</param>
        </plugin>
        <plugin id="local" type="runner" load="galaxy.jobs.runners.local:LocalJobRunner" workers="2"/>
    </plugins>
    <handlers default="handlers">
        <handler id="handler0" tags="handlers"/>
        <handler id="handler1" tags="handlers"/>
    </handlers>
    <destinations default="slurm_cluster">
        <destination id="slurm_cluster" runner="slurm">
            <param id="nativeSpecification">-p work -n 8</param>
            <param id="embed_metadata_in_job">False</param>
            <env file="/export/galaxy-central/.venv/bin/activate" />
        </destination>
        <destination id="local" runner="local"/>
    </destinations>
    <tools>
        <tool id="upload1" destination="local"/>
    </tools>
    <limits>
    </limits>
</job_conf>
'''

f = open('tmp/job_conf.xml', 'w')
f.write(job_conf)
f.close()

!scp -F tmp/ssh_config.ga tmp/job_conf.xml galaxy-server:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker cp job_conf.xml galaxy:/etc/galaxy/

## Install a galaxy tool (HISAT2)

Galaxy で HISAT2 を実行するためのツール定義を作成し、Galaxyのツール設定に追加する。  
  * 今回のハンズオン向けに、起動オプション等は省略している。

In [ ]:
# sample tool 定義を作成
tool_config = '''\
<tool id="hisat2" name="HISAT2" version="2.1.0">
    <description>A fast and sensitive alignment program</description>
    <stdio>
        <regex level="fatal" match="hisat2-align exited with value 1" source="both" />
        <regex level="fatal" match="hisat2: not found" source="both" />
        <exit_code range="1:" />
    </stdio>
    <version_command>hisat2 --version</version_command>
    <command><![CDATA[
hisat2 -p 8 -x /export/genome/hisat2-index/hg19 --dta --dta-cufflinks $input1 | samtools view -bS - | samtools sort -o - tmp > $mapped_reads;
]]>
  </command>
  
  <inputs>
    <param name="input1" format="fastq" type="data" label="fastq" />
  </inputs>

  <outputs>
     <data name="mapped_reads" format="bam" label="${tool.name} on ${on_string}: mapped reads" from_work_dir="Aligned.sortedByCoord.out.bam"/>
  </outputs>

  <help></help>

</tool>
'''

f = open('tmp/hisat2.xml', 'w')
f.write(tool_config)
f.close()

!scp -F tmp/ssh_config.ga tmp/hisat2.xml galaxy-server:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy mkdir -p tools/test
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker cp hisat2.xml galaxy:/galaxy-central/tools/test/

Galaxy のツール定義リスト設定ファイル `tool_conf.xml` の末尾に HISAT2 用のツール定義を追加する。

（修正前）
```
root@master:~# tail /export/galaxy-central/config/tool_conf.xml.sample 
  <section id="stats" name="Statistics">
    <tool file="stats/gsummary.xml" />
    <tool file="filters/uniq.xml" />
  </section>
  <section id="plots" name="Graph/Display Data">
    <tool file="plotting/bar_chart.xml" />
    <tool file="plotting/boxplot.xml" />
    <tool file="maf/vcf_to_maf_customtrack.xml" />
  </section>
</toolbox>
```
----
（修正後）
```
  <section id="stats" name="Statistics">
    <tool file="stats/gsummary.xml" />
    <tool file="filters/uniq.xml" />
  </section>
  <section id="plots" name="Graph/Display Data">
    <tool file="plotting/bar_chart.xml" />
    <tool file="plotting/boxplot.xml" />
    <tool file="maf/vcf_to_maf_customtrack.xml" />
  </section>
  <section id="test" name="RNA Seq">
    <tool file="test/hisat2.xml" />
  </section>
</toolbox>
```

In [ ]:
update_tool_config = '''\
sed -i 's|</toolbox>|<section id="test" name="RNA Seq"><tool file="test/hisat2.xml" /></section></toolbox>|' /export/galaxy-central/config/tool_conf.xml.sample
'''

f = open('tmp/tool_conf_update.sh', 'w')
f.write(update_tool_config)
f.close()

!scp -F tmp/ssh_config.ga tmp/tool_conf_update.sh galaxy-server:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker cp tool_conf_update.sh galaxy:/tmp/
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec galaxy sh /tmp/tool_conf_update.sh

## Restart Galaxy server

ジョブ実行設定を反映させるために、Galaxy Serverを再起動する。

In [ ]:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy supervisorctl restart galaxy:

# Access to Galaxy web

[Galaxy Web](/galaxy/)
  * Username `admin@galaxy.org`, Password `admin` でログイン


## Get sample data (FASTQ file) 
Galaxy Web UI の Get Data > Upload File メニューから下図の手順で取得する。

次のダウンロード先URLを入力 ⇒ `https://s3-ap-northeast-1.amazonaws.com/vcp-hands-on/fastq/SRR934273_1.fastq`

![get-fastq-data](media/get-fastq-data.png)

## Execute HISAT2

![exec-hisat2](media/exec-hisat2.png)

## Show  Grafana graphs

[Grafanaダッシュボード](/grafana/dashboard/db/vcp1204?orgId=1&refresh=5s&from=now-15m&to=now)
  * User `admin`, Password `admin` でログイン

# Check status of the SLURM cluster

Galaxy Web で HISAT2 を実行すると、SLURM のノード使用状況が変化することが確認できる。（必要に応じて）

In [32]:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy sinfo --long

Mon Dec  4 07:58:50 2017
PARTITION AVAIL  TIMELIMIT   JOB_SIZE ROOT SHARE     GROUPS  NODES       STATE NODELIST
work*        up   infinite 1-infinite   no YES:4        all      1   allocated c1
work*        up   infinite 1-infinite   no YES:4        all      1        idle c2


# Cleanup VC

ここまでで作成した Galaxy 用 VC を丸ごと破棄することができる。

** ※ 今回のハンズオンでは Cleanup を実行せず、環境をそのまま放置していただいて構いません。**

In [40]:
v.cleanup()

# Appendix

## Scale-out a virtual cloud for Galaxy + SLURM cluster

[SC17](https://sc17.supercomputing.org/) にて、Galaxyクラスタを複数クラウドにまたがってスケールアウトさせるデモ環境を構築した。
- NIIクラウドで構築したGalaxyクラスタ環境の VC (Virtual Cloud) をAWSに拡張する。
- Galaxyのジョブ実行先となる計算ノード群のSLURMクラスタにAWS上のノードを追加する。

![galaxy-slurm-cluster](media/SC17-galaxy-slurm.png)

| VC Node | Cloud (type) | Role |
|:--------|:-------------|:----------------------|
| 1 | NII Cloud (cpu:20 ram:128G) | NFS / DNS / Galaxy / SLURM Controller |
| 2 | NII Cloud (cpu:20 ram:128G) | SLURM Node \#1 |
| 3 | NII Cloud (cpu:20 ram:128G) | SLURM Node \#2 |
| 4 | Amazon Web Services (m4.4xlarge)     | SLURM Node \#3 |
| 5 | Amazon Web Services (m4.4xlarge)     | SLURM Node \#4 |


In [35]:
!ssh -F tmp/ssh_config.ga galaxy-server /usr/local/bin/docker exec -i galaxy sinfo --long

Wed Dec  6 22:58:14 2017
PARTITION AVAIL  TIMELIMIT   JOB_SIZE ROOT SHARE     GROUPS  NODES       STATE NODELIST
work*        up   infinite 1-infinite   no YES:4        all      2        idle c[1-2]
